In [1]:
import os
os.chdir("../..")

In [2]:
assert "struc2vec" in os.listdir()

In [16]:
from struc2vec.src import struc2vec as s2v
import networkx as nx
import pandas as pd
# from gensim.models import Word2Vec
import numpy as np
# from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


In [5]:
os.chdir("structuralNetworkMigration")
assert "data" in os.listdir()

## Load data

First load each graph

In [6]:
G = {"0":None,"25":None,"50":None, "75":None}

for perc, graph in G.items():
    G[perc] =  nx.read_gexf(f"data/Graphs/bynode_{perc}_directed.gexf")

Create struc2vec objects

In [8]:
s2vG_dict = {}
for perc, graph in G.items():
    s2vG_dict[perc] = s2v.struc2vec(graph)

Load contextgraphs

In [ ]:
for perc, s2vG in s2vG_dict.items():
    s2vG.getMultiLevelGraph()
    try:
        for i, G in s2vG.G_ML.items():
            nx.write_gexf(G, f"data/GML/{perc}/{i}_fromnode.gexf")
    except:
        continue

In [ ]:
    # for i, G in cur_s2vG.G_ML.items():
    #     nx.write_gexf(G, f"GML/{perc}/{i}.gexf")

In [ ]:
# for perc, s2vG in s2vG_dict.items():
#     s2vG.getMultiLevelGraph(path=f"data/GML/{perc}")

## Final embedding

In [17]:
walks = []
for perc, s2vG in s2vG_dict.items():
    n_walk = 500
    if (perc == "25")|(perc=="50"):
        n_walk = 1000
    walks_ = s2vG.getRandomWalks(number_of_walks = n_walk, walk_length=80)
    walks.extend(walks_)

# model = Word2Vec(walks, vector_size=75)
# embeddings = {node: model.wv[node] for node in G["0"].nodes}
# embeddings_values = [x for x in embeddings.values()]
# X = np.array(embeddings_values)

ValueError: invalid literal for int() with base 10: np.str_('LTU')

In [23]:
pd.DataFrame(X).to_csv("data/embeddings/from_graph_thresholds.csv")